# Fine-tuning di modelli su OpenAI

In [5]:
import os
from langchain_openai import ChatOpenAI
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import tiktoken
import numpy as np
from collections import defaultdict
import json

In [6]:
# caricamento dati
data = pd.read_excel("data/welcomes.xlsx")  # pip install openpyxl

data.head()

,Unnamed: 0,HumanMessage,AIMessage
0,0,"#welcome_message# Data Analyst , Luciana Romano","Cara Luciana Romano,\n\nSiamo lieti di darti i..."
1,1,"#welcome_message# Data Analyst , xxx Romano","Gentile xxx Romano,\n\nTi diamo il benvenuto a..."
2,2,"#welcome_message# Data Analyst , asd Smith","Caro asd Smith,\n\nSiamo lieti di darti il ben..."
3,3,"#welcome_message# Python Developer , Axe","Ciao Axe,\n\nSiamo entusiasti di darti il benb..."
4,4,"#welcome_message# Data Analyst , Jones giovanna","Cara Giovanna Jones,\n\nSiamo lieti di darti i..."


In [7]:
test = []
for idx, row in data.iterrows():
    test.append({
        "messages": [{
            "role": "system",
            "content": "#WlcmMsgs#"
        }, {
            "role": "user",
            "content": row.values[1]
        }, {
            "role": "assistant",
            "content": row.values[2]
        }]
    })

In [8]:
print(test)

[{'messages': [{'role': 'system', 'content': '#WlcmMsgs#'}, {'role': 'user', 'content': '#welcome_message# Data Analyst , Luciana Romano'}, {'role': 'assistant', 'content': "Cara Luciana Romano,\n\nSiamo lieti di darti il benvenuto nella tua sessione di valutazione personalizzata su tecniche, metologie e argomenti legati alla professione di Data Analyst; questa sessione servirà a valutare ed esaltare le tue competenze in questo campo.\n\nDurante questa valutazione, avrai l'opportunità di dimostrare le tue abilità e identificare le aree in cui concentrare la tua crescita professionale. Ricorda di dare risposte brevi e precise.\n\nNon vediamo l'ora di vedere i tuoi progressi e ti auguriamo buona fortuna per questa entusiasmante sfida.\n\nCominciamo!"}]}, {'messages': [{'role': 'system', 'content': '#WlcmMsgs#'}, {'role': 'user', 'content': '#welcome_message# Data Analyst , xxx Romano'}, {'role': 'assistant', 'content': "Gentile xxx Romano,\n\nTi diamo il benvenuto alla tua sessione perso

In [9]:
# creazione di un file JSONL partendo dal nostro file Excel

with open("data/welcomes.jsonl", 'w') as f:
    for item in test:
        f.write(json.dumps(item) + "\n")

In [10]:
# recupero file JSONL

data_path = "data/welcomes.jsonl"

# caricamento dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# stampa di statistiche elementari sul dataset
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 29
First example:
{'role': 'system', 'content': '#WlcmMsgs#'}
{'role': 'user', 'content': '#welcome_message# Data Analyst , Luciana Romano'}
{'role': 'assistant', 'content': "Cara Luciana Romano,\n\nSiamo lieti di darti il benvenuto nella tua sessione di valutazione personalizzata su tecniche, metologie e argomenti legati alla professione di Data Analyst; questa sessione servirà a valutare ed esaltare le tue competenze in questo campo.\n\nDurante questa valutazione, avrai l'opportunità di dimostrare le tue abilità e identificare le aree in cui concentrare la tua crescita professionale. Ricorda di dare risposte brevi e precise.\n\nNon vediamo l'ora di vedere i tuoi progressi e ti auguriamo buona fortuna per questa entusiasmante sfida.\n\nCominciamo!"}


In [11]:
# check conformità / errori

# Dizionario per tenere traccia di eventuali errori di formato
format_errors = defaultdict(int)

# Itera su ogni esempio nel dataset
for ex in dataset:
    # Controlla se l'esempio è un dizionario
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1  # Incrementa il contatore per errori di tipo dati
        continue  # Passa all'esempio successivo

    # Verifica se l'esempio contiene la chiave "messages"
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1  # Registra l'errore se manca la lista dei messaggi
        continue

    # Controlla la validità di ciascun messaggio nella lista
    for message in messages:
        # Verifica la presenza delle chiavi obbligatorie "role" e "content"
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        # Controlla se ci sono chiavi sconosciute nel messaggio
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1

        # Controlla se il valore della chiave "role" è valido
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        # Estrae il contenuto del messaggio e la chiamata di funzione
        content = message.get("content", None)
        function_call = message.get("function_call", None)

        # Verifica che il messaggio abbia contenuto valido o una chiamata di funzione
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    # Controlla se almeno un messaggio ha il ruolo "assistant"
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

# Stampa il riepilogo degli errori trovati
if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [12]:
# funzioni per il conteggio dei token e la creazione di statistiche sul dataset

encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    '''
    Durante il fine-tuning, spesso vengono utilizzati token speciali per indicare l'inizio `<|startoftext|>` 
    e la fine `<|endoftext|>` di una sequenza. Questi token aiutano il modello a comprendere meglio 
    i limiti dei dati di addestramento.
    
    In questo metodo `tokens_per_message` serve per considerare i 3 token utilizzati nel fine-tuning
    per i tag di inizio e fine esempio.
    '''
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message  # Ogni messaggio ha un overhead fisso di tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += tokens_per_message
    return num_tokens


def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens


def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")


In [13]:
MAX_TOKENS_PER_EXAMPLE = 4096

n_missing_system = 0
n_missing_user = 0
n_messages = []
tot_tokens_per_example = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    tot_tokens_per_example.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(tot_tokens_per_example, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")

n_too_long = sum(l > MAX_TOKENS_PER_EXAMPLE for l in tot_tokens_per_example)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 151, 231
mean / median: 191.13793103448276, 196.0
p5 / p95: 171.6, 207.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 119, 194
mean / median: 158.31034482758622, 163.0
p5 / p95: 137.8, 172.8

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [14]:
# Stima dei token necessari per il fine-tuning

# valori di default suggeriti da OpenAI
TARGET_EPOCHS = 15
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in tot_tokens_per_example)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~5543 tokens that will be charged for during training
By default, you'll train for 15 epochs on this dataset
By default, you'll be charged for ~83145 tokens


### test modello fine-tunato

In [15]:
welcomes_model = ChatOpenAI(
    model="ft:gpt-4o-mini-2024-07-18:data-masters-s-r-l:test-fine-tuning:CW2pBv4f",
    temperature=0.,
    openai_api_key=os.getenv("openai_key"),
    max_tokens=1500,
    request_timeout=35
)

In [16]:
welcomes_model.invoke([
    SystemMessage(
        content="#WlcmMsgs#"),
    HumanMessage(
        content="#welcome_message# musicista , Rosa"),
])

AIMessage(content="Cara Rosa,\n\nTi diamo il benvenuto alla tua sessione di valutazione personalizzata per musicisti. Questa sessione è stata progettata per valutare le tue competenze musicali e per identificare le aree in cui potresti voler concentrare i tuoi futuri studi.\n\nRicorda, questa è un'opportunità per mostrare le tue abilità e per scoprire nuove aree di crescita. Siamo entusiasti di accompagnarti in questo viaggio musicale e non vediamo l'ora di vedere come ti esibirai.\n\nIniziamo con la prima domanda... Buona fortuna, Rosa!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 25, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'ft:gpt-4o-mini-2024-07-18:data-masters-s-r-l:test-fine-tuning:CW2